## Fine-Tuning and Evaluating LLMs with SageMaker Pipelines and MLflow

Running hundreds of experiments, comparing the results, and keeping a track of the ML lifecycle can become very complex. This is where MLflow can help streamline the ML lifecycle, from data preparation to model deployment. By integrating MLflow into your LLM workflow, you can efficiently manage experiment tracking, model versioning, and deployment, providing reproducibility. With MLflow, you can track and compare the performance of multiple LLM experiments, identify the best-performing models, and deploy them to production environments with confidence. 

You can create workflows with SageMaker Pipelines that enable you to prepare data, fine-tune models, and evaluate model performance with simple Python code for each step. 

Now you can use SageMaker managed MLflow to run LLM fine-tuning and evaluation experiments at scale. Specifically:

- MLflow can manage tracking of fine-tuning experiments, comparing evaluation results of different runs, model versioning, deployment, and configuration (such as data and hyperparameters)
- SageMaker Pipelines can orchestrate multiple experiments based on the experiment configuration 
  

The following figure shows the overview of the solution.
![](./ml-16670-arch-with-mlflow.png)

## Prerequisites 
Before you begin, make sure you have the following prerequisites in place:

- [HuggingFace access token](https://huggingface.co/docs/hub/en/security-tokens) – You need a HuggingFace login token to access the gated Llama 3.2 model and datasets used in this post.

- Once you have your HuggingFace access token, navigate to the **steps/finetune_llama3b_hf.py** and update the **'hf_token'** parameter with your access token to download the Llama model for fine-tuning.

### 1. Setup and Dependencies
Restart the kernel after executing below cells

In [ ]:
%pip install -r requirements.txt --upgrade
%pip install -q -U python-dotenv

In [ ]:
%load_ext autoreload
%autoreload 2

**Importing Libraries and Setting Up Environment**

This part imports all necessary Python modules. It includes SageMaker-specific imports for pipeline creation and execution, as well as user-defined functions for the pipeline steps like finetune_llama3b_hf and preprocess_llama3.

In [ ]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.function_step import step
from steps.finetune_llama3b_hf import finetune_llama3b
from steps.preprocess_llama3 import preprocess
from steps.evaluation_mlflow import evaluation
from steps.utils import create_training_job_name
import os

os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

### 2. SageMaker Session and IAM Role

`get_execution_role()`: Retrieves the IAM role that SageMaker will use to access AWS resources. This role needs appropriate permissions for tasks like accessing S3 buckets and creating SageMaker resources.

In [ ]:
import boto3

try:
    role = sagemaker.get_execution_role()
    print(role)
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

sess = sagemaker.Session()

### 3. Configuration

**Training Configuration**

The train_config dictionary is comprehensive, including:

Experiment naming for tracking purposes
Model specifications (ID, version, name)
Infrastructure details (instance types and counts for fine-tuning and deployment)
Training hyperparameters (epochs, batch size)

This configuration allows for easy adjustment of the training process without changing the core pipeline code.

In [ ]:
train_config = {
    "experiment_name": "all_target_modules_1K",
    "model_id": "meta-llama/Llama-3.2-3B",
    "model_name": "llama-32-3b",
    "endpoint_name": "llama-32-3b",
    "finetune_instance_type": "ml.g5.12xlarge",
    "finetune_num_instances": 1,
    "instance_type": "ml.g5.12xlarge",
    "num_instances": 1,
    "epoch": 1,
    "per_device_train_batch_size": 4,
}

**LoRA Parameters**

Low-Rank Adaptation (LoRA) is an efficient fine-tuning technique for large language models. The parameters here (lora_r, lora_alpha, lora_dropout) control the behavior of LoRA during fine-tuning, affecting the trade-off between model performance and computational efficiency.

In [ ]:
lora_params = {"lora_r": 8, "lora_alpha": 16, "lora_dropout": 0.05}

### 4. MLflow Setup

MLflow integration is crucial for experiment tracking and management. **Update the ARN for the MLflow tracking server.**

mlflow_arn: The ARN for the MLflow tracking server. You can get this ARN from SageMaker Studio UI. This allows the pipeline to log metrics, parameters, and artifacts to a central location.

experiment_name: give appropriate name for experimentation

In [ ]:
mlflow_arn = "<ENTER MLflow TRACKING SERVER ARN>"  # fill MLflow tracking server ARN
experiment_name = "sm-pipelines-finetuning"

### 5. Dataset Configuration

For the purpose of fine tuning and evaluation we are going too use `HuggingFaceH4/no_robots` dataset

In [ ]:
dataset_name = "HuggingFaceH4/no_robots"

### 6. Pipeline Steps

This section defines the core components of the SageMaker pipeline.

In [ ]:
from sagemaker.workflow.parameters import ParameterString
import json

In [ ]:
lora_config = ParameterString(name="lora_config", default_value=json.dumps(lora_params))

**Preprocessing Step**

This step handles data preparation. We are going to prepare data for training and evaluation. We will log this data in MLflow

In [ ]:
pipeline_name = "fmops-training-evaulation-pipeline-mlflow"

default_bucket = sagemaker.Session().default_bucket()
main_data_path = f"s3://{default_bucket}"
evaluation_data_path = (
    main_data_path
    + "/datasets/hf_no_robots/evaluation/automatic_small/dataset_evaluation_small.jsonl"
)
output_data_path = main_data_path + "/datasets/hf_no_robots/output_" + pipeline_name

# You can add your own evaluation dataset code into this step
preprocess_step_ret = step(preprocess, name="preprocess")(
    default_bucket,
    dataset_name,
    train_sample=100,
    eval_sample=100,
    mlflow_arn=mlflow_arn,
    experiment_name=experiment_name,
    run_name=ExecutionVariables.PIPELINE_EXECUTION_ID,
)

print("The pipeline name is " + pipeline_name)
# Mark the name of this bucket for reviewing the artifacts generated by this pipeline at the end of the execution
print("Output S3 bucket: " + output_data_path)

**Fine-tuning Step**

This is where the actual model adaptation occurs. The step takes the preprocessed data and applies it to fine-tune the base LLM (in this case, a Llama model). It incorporates the LoRA technique for efficient adaptation.

In [ ]:
finetune_ret = step(finetune_llama3b, name="finetune_llama3b_instruction")(
    preprocess_step_ret,
    train_config,
    lora_config,
    role,
    mlflow_arn,
    experiment_name,
    ExecutionVariables.PIPELINE_EXECUTION_ID,
)

**Evaluation Step**

After fine-tuning, this step assesses the model's performance. It uses built-in evaluation function in MLflow to evaluate metrices like toxicity, exact_match etc:

It will then log the results in MLflow

In [ ]:
evaluate_finetuned_llama3b_instruction_mlflow = step(
    evaluation,
    name="evaluate_finetuned_llama3b_instr",
    # keep_alive_period_in_seconds=1200,
    instance_type="ml.g5.12xlarge",
    volume_size=100,
)(train_config, preprocess_step_ret, finetune_ret, mlflow_arn, experiment_name, "")

### 7. Pipeline Creation and Execution

This final section brings all the components together into an executable pipeline.

**Creating the Pipeline**

The pipeline object is created with all defined steps. The lora_config is passed as a parameter, allowing for easy modification of LoRA settings between runs.

In [ ]:
from sagemaker import get_execution_role

pipeline = Pipeline(
    name=pipeline_name,
    steps=[evaluate_finetuned_llama3b_instruction_mlflow],
    parameters=[lora_config],
)

**Upserting the Pipeline**

This step either creates a new pipeline in SageMaker or updates an existing one with the same name. It's a key part of the MLOps process, allowing for iterative refinement of the pipeline.

In [ ]:
pipeline.upsert(role)

**Starting the Pipeline Execution**

This command kicks off the actual execution of the pipeline in SageMaker. From this point, SageMaker will orchestrate the execution of each step, managing resources and data flow between steps.

In [ ]:
execution1 = pipeline.start()

# Clean up

In [ ]:
sagemaker_client = boto3.client("sagemaker")
response = sagemaker_client.delete_pipeline(
    PipelineName=pipeline_name,
)